In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
10Y Crypto Averages + Key Levels (Live Overlay)
- Fetch daily OHLCV (up to ~10 years where available) via ccxt
- Compute rolling quantile-based support/resistance (avg bands, 90D)
- Compute 90D average price, bull/bear regime (SMA200), annualized volatility
- Summarize average bull/bear run lengths
- CHART 1: Price + Avg Price/Support/Resistance with KEY S/R LEVELS highlighted
- CHART 2: Live-price overlay vs those KEY S/R LEVELS for current trading

Notebook-safe: uses parse_known_args() and prompts when args are missing.
Robust symbol mapping (e.g., 'BTC' -> BTC/USDT/BTC/USD/etc., Kraken XBT alias).
"""
# SECURITY NOTE:
# This tool uses only public market data from exchanges via ccxt.
# It does not require API keys and does not place trades.
# Charts are for research/education only, not trading advice.


import math
import sys
import argparse
import time
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    import ccxt  # pip install ccxt
except Exception as e:
    raise SystemExit("ccxt is required. Install with: pip install ccxt") from e


PREF_QUOTES = ["USDT", "USD", "USDC", "BTC", "ETH"]


# --------------------------- symbol helpers ---------------------------

def _normalize_user_symbol(s: str) -> str:
    s = (s or "").strip().upper().replace(" ", "")
    s = s.replace("-", "/")
    return s

def _find_best_symbol(ex, user_sym: str) -> str:
    """
    Map user input to a concrete market symbol on the exchange.
    Accepts 'BTC', 'BTC/USDT', 'btc-usdt', etc.
    Prefers spot markets. Handles Kraken XBT alias for BTC.
    """
    ex.load_markets()
    markets = ex.markets or {}
    all_symbols = list(markets.keys())

    # If user included a quote, try exact first
    if "/" in user_sym:
        if user_sym in markets:
            return user_sym
        base, quote = user_sym.split("/")
        if base == "BTC" and f"XBT/{quote}" in markets:  # Kraken alias
            return f"XBT/{quote}"
        if quote == "USDT" and f"{base}/USD" in markets:
            return f"{base}/USD"
        if quote == "USD" and f"{base}/USDT" in markets:
            return f"{base}/USDT"

    # If no slash, try preferred quotes in order
    if "/" not in user_sym:
        base = user_sym
        base_alt = "XBT" if (base == "BTC" and any(sym.startswith("XBT/") for sym in all_symbols)) else base
        for q in PREF_QUOTES:
            cand = f"{base_alt}/{q}"
            if cand in markets:
                return cand
            cand2 = f"{base}/{q}"
            if cand2 in markets:
                return cand2

    # Last resort: scan symbols that start with base and pick a spot one (if type available)
    base = user_sym.split("/")[0] if "/" in user_sym else user_sym
    base_alt = "XBT" if base == "BTC" else base
    candidates = [s for s in all_symbols if s.startswith(base_alt + "/") or s.startswith(base + "/")]
    if candidates:
        def rank(sym):
            m = markets.get(sym, {})
            t = m.get("type") or m.get("spot")
            if t is True or t == "spot":
                return 0
            if t == "future" or t == "swap" or m.get("swap"):
                return 1
            return 2
        candidates.sort(key=lambda x: (rank(x), PREF_QUOTES.index(x.split("/")[1]) if x.split("/")[1] in PREF_QUOTES else 99))
        return candidates[0]

    raise RuntimeError(f"Could not map '{user_sym}' to a tradable market on {ex.id}. Try e.g. BTC/USDT.")


# --------------------------- data fetch ---------------------------

def fetch_ohlcv_daily(exchange_id: str, user_symbol: str, since_ms: int) -> pd.DataFrame:
    ex = getattr(ccxt, exchange_id)({
        "enableRateLimit": True,
        "options": {"defaultType": "spot"}
    })

    user_symbol = _normalize_user_symbol(user_symbol)
    sym = _find_best_symbol(ex, user_symbol)

    tf = "1d"
    all_rows, limit, since = [], 1000, since_ms

    while True:
        batch = ex.fetch_ohlcv(sym, timeframe=tf, since=since, limit=limit)
        if not batch:
            break
        all_rows += batch
        if len(batch) < limit:
            break
        since = batch[-1][0] + 24 * 60 * 60 * 1000
        time.sleep((ex.rateLimit or 200) / 1000.0)

    if not all_rows:
        raise RuntimeError(f"No OHLCV returned for {sym} on {exchange_id}. Try a different exchange/symbol.")

    df = pd.DataFrame(all_rows, columns=["ts", "open", "high", "low", "close", "volume"])
    df["ts"] = pd.to_datetime(df["ts"], unit="ms", utc=True)
    df = df.dropna().sort_values("ts").reset_index(drop=True)
    df.attrs["resolved_symbol"] = sym
    df.attrs["exchange"] = exchange_id
    return df

def fetch_live_price(exchange_id: str, resolved_symbol: str) -> float | None:
    try:
        ex = getattr(ccxt, exchange_id)({"enableRateLimit": True, "options": {"defaultType": "spot"}})
        t = ex.fetch_ticker(resolved_symbol)
        last = t.get("last") or t.get("close")
        return float(last) if last is not None else None
    except Exception:
        return None


# --------------------------- indicators ---------------------------

def rolling_quantiles(s: pd.Series, q: float, win: int) -> pd.Series:
    return s.rolling(win, min_periods=max(10, win // 2)).quantile(q)

def rolling_ann_vol(s: pd.Series, win: int, periods_per_year: int = 365) -> pd.Series:
    r = s.pct_change()
    vol = r.rolling(win, min_periods=max(10, win // 2)).std()
    return vol * math.sqrt(periods_per_year)

def bull_bear_regime(close: pd.Series, long_ma: int = 200) -> pd.Series:
    sma = close.rolling(long_ma, min_periods=max(20, long_ma // 2)).mean()
    return (close > sma).astype(int).replace(0, -1)  # 1 bull, -1 bear

def average_run_lengths(regime: pd.Series) -> dict:
    runs, cur, length = [], None, 0
    for v in regime.dropna():
        if cur is None:
            cur, length = v, 1
        elif v == cur:
            length += 1
        else:
            runs.append((cur, length))
            cur, length = v, 1
    if cur is not None:
        runs.append((cur, length))
    bull = [l for (r, l) in runs if r == 1]
    bear = [l for (r, l) in runs if r == -1]
    return {
        "bull_avg_days": float(np.mean(bull)) if bull else float("nan"),
        "bear_avg_days": float(np.mean(bear)) if bear else float("nan"),
        "bull_runs": len(bull),
        "bear_runs": len(bear),
    }

def build_summary(df: pd.DataFrame, win: int = 90) -> pd.DataFrame:
    d = df.copy()
    d.set_index("ts", inplace=True)
    d["avg_price"]      = d["close"].rolling(win, min_periods=max(10, win // 2)).mean()
    d["avg_support"]    = rolling_quantiles(d["close"], 0.20, win)
    d["avg_resistance"] = rolling_quantiles(d["close"], 0.80, win)
    d["regime"]         = bull_bear_regime(d["close"], long_ma=200)   # 1 bull, -1 bear
    d["risk_vol"]       = rolling_ann_vol(d["close"], win)            # annualized vol (secondary axis)
    return d.reset_index()


# --------------------------- key level extraction ---------------------------

def key_levels_from_bands(d: pd.DataFrame, n_each: int = 3) -> dict:
    """
    Derive multiple 'key' S/R levels from the historical average bands.
    We use robust percentiles to avoid overfitting:
      - Supports: p10, p20, p30 of avg_support (plus its mean)
      - Resistances: p70, p80, p90 of avg_resistance (plus its mean)
      - Value area: median (p50) of avg_price
    """
    ds = d.dropna(subset=["avg_support", "avg_resistance", "avg_price"])
    if ds.empty:
        return {"supports": [], "resistances": [], "value": []}

    sup_vals = ds["avg_support"].values
    res_vals = ds["avg_resistance"].values
    val_vals = ds["avg_price"].values

    # Supports
    sup_percs = [10, 20, 30]
    sup = [float(np.nanpercentile(sup_vals, p)) for p in sup_percs[:n_each]]
    sup.append(float(np.nanmean(sup_vals)))  # average support

    # Resistances
    res_percs = [90, 80, 70]
    res = [float(np.nanpercentile(res_vals, p)) for p in res_percs[:n_each]]
    res.append(float(np.nanmean(res_vals)))  # average resistance

    # Value area mid
    value = [float(np.nanmedian(val_vals)), float(np.nanmean(val_vals))]

    # Deduplicate and sort
    sup = sorted(list({round(x, 12) for x in sup if np.isfinite(x)}))
    res = sorted(list({round(x, 12) for x in res if np.isfinite(x)}))
    value = sorted(list({round(x, 12) for x in value if np.isfinite(x)}))

    return {"supports": sup, "resistances": res, "value": value}


# --------------------------- plotting ---------------------------

def _fmt(ax):
    ax.grid(True, alpha=0.25)
    for spine in ("top", "right"):
        ax.spines[spine].set_visible(False)

def plot_average_chart_with_levels(d: pd.DataFrame, symbol_shown: str, levels: dict):
    """
    CHART 1: Price + Avg bands + highlighted key S/R horizontal lines
    """
    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(d["ts"], d["close"],          label="Close", linewidth=1.1)
    ax1.plot(d["ts"], d["avg_price"],      label="Avg Price (90D)", linewidth=1.2)
    ax1.plot(d["ts"], d["avg_support"],    label="Avg Support (q20, 90D)", linewidth=1.2)
    ax1.plot(d["ts"], d["avg_resistance"], label="Avg Resistance (q80, 90D)", linewidth=1.2)

    # Highlight key supports (greens) & resistances (reds)
    for y in levels.get("supports", []):
        ax1.axhline(y, color="green", alpha=0.25, linewidth=1.5)
        ax1.text(d["ts"].iloc[-1], y, f"  S {y:.4f}", va="center", ha="left", color="green", alpha=0.7)
    for y in levels.get("resistances", []):
        ax1.axhline(y, color="red", alpha=0.25, linewidth=1.5)
        ax1.text(d["ts"].iloc[-1], y, f"  R {y:.4f}", va="center", ha="left", color="red", alpha=0.7)

    # Value area midlines (blue)
    for y in levels.get("value", []):
        ax1.axhline(y, color="tab:blue", alpha=0.2, linewidth=1.2, linestyle="--")
        ax1.text(d["ts"].iloc[-1], y, f"  VA {y:.4f}", va="center", ha="left", color="tab:blue", alpha=0.7)

    ax1.set_xlabel("Date (UTC)")
    ax1.set_ylabel("Price")
    ax1.legend(loc="upper left")

    # Secondary axis: risk (annualized vol)
    ax2 = ax1.twinx()
    ax2.plot(d["ts"], d["risk_vol"], label="Risk (Ann Vol, 90D)", color="gray", alpha=0.6)
    ax2.set_ylabel("Annualized Volatility")
    ax2.legend(loc="upper right")

    _fmt(ax1); _fmt(ax2)
    plt.title(f"{symbol_shown} — Average Bands with Key S/R Levels")
    plt.tight_layout()
    plt.show()

def plot_live_overlay(levels: dict, live_price: float | None, last_close: float, symbol_shown: str):
    """
    CHART 2: Horizontal key S/R levels + markers for last close and live price.
    Gives a clean picture of where price sits relative to historically 'key' averages.
    """
    lines = []
    labels = []
    colors = []

    for y in levels.get("supports", []):
        lines.append(y); labels.append(f"S {y:.4f}"); colors.append("green")
    for y in levels.get("value", []):
        lines.append(y); labels.append(f"VA {y:.4f}"); colors.append("tab:blue")
    for y in levels.get("resistances", []):
        lines.append(y); labels.append(f"R {y:.4f}"); colors.append("red")

    if not lines:
        print("No key levels to plot.")
        return

    y_min, y_max = min(lines + [last_close, live_price if live_price else last_close]), max(lines + [last_close, live_price if live_price else last_close])
    pad = (y_max - y_min) * 0.10 if (y_max - y_min) > 0 else max(1.0, y_max * 0.05)
    y_min -= pad; y_max += pad

    fig, ax = plt.subplots(figsize=(10, 6))

    # draw horizontal lines
    for y, lab, c in zip(lines, labels, colors):
        ax.axhline(y, color=c, alpha=0.35, linewidth=2.0)
        ax.text(0.02, (y - y_min) / (y_max - y_min), lab, transform=ax.transAxes, va="center", ha="left", color=c, alpha=0.9)

    # markers for last close and live price
    ax.plot([0.5], [last_close], marker="o", color="black", label=f"Last close {last_close:.4f}")
    if live_price is not None:
        ax.plot([0.6], [live_price], marker="X", color="purple", label=f"Live {live_price:.4f}")

    ax.set_ylim([y_min, y_max])
    ax.set_xlim([0, 1])
    ax.set_xticks([])
    ax.set_ylabel("Price")
    ax.legend(loc="upper right")
    _fmt(ax)
    plt.title(f"{symbol_shown} — Live Price vs Key Average S/R Levels")
    plt.tight_layout()
    plt.show()


# --------------------------- CLI / user input ---------------------------

def get_params_from_user(default_symbol="BTC/USDT", default_exchange="binance", default_years=10):
    try:
        sym = input(f"Enter symbol (e.g., BTC or BTC/USDT) [{default_symbol}]: ").strip() or default_symbol
        ex  = input(f"Enter exchange id (ccxt, e.g., binance, bybit, kraken) [{default_exchange}]: ").strip() or default_exchange
        yrs_in = input(f"Years of history (max available) [{default_years}]: ").strip()
        yrs = int(yrs_in) if yrs_in else default_years
    except Exception:
        sym, ex, yrs = default_symbol, default_exchange, default_years
    return _normalize_user_symbol(sym), ex.lower(), max(1, yrs)

def parse_args_or_prompt():
    parser = argparse.ArgumentParser(add_help=True)
    parser.add_argument("--symbol",   type=str, default=None, help="e.g., BTC or BTC/USDT")
    parser.add_argument("--exchange", type=str, default=None, help="ccxt exchange id, e.g., binance")
    parser.add_argument("--years",    type=int, default=None, help="years back, default 10")
    args, _unknown = parser.parse_known_args()

    sym, ex, yrs = args.symbol, args.exchange, args.years
    if not sym or not ex or not yrs:
        sym, ex, yrs = get_params_from_user(
            default_symbol=sym or "BTC/USDT",
            default_exchange=ex or "binance",
            default_years=yrs or 10
        )
    else:
        sym = _normalize_user_symbol(sym)
        ex = ex.lower()
        yrs = max(1, yrs)
    return sym, ex, yrs


# --------------------------- main ---------------------------

def main():
    symbol, exchange_id, years = parse_args_or_prompt()
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=365 * years + 30)
    since_ms = int(start.timestamp() * 1000)

    print(f"\nFetching {symbol} daily candles from {exchange_id} since {start.date()}…")
    df = fetch_ohlcv_daily(exchange_id, symbol, since_ms)
    resolved = df.attrs.get("resolved_symbol", symbol)
    print(f"Resolved market: {resolved} on {exchange_id}")

    have_days = (df["ts"].max() - df["ts"].min()).days
    target_days = (end - start).days
    if have_days < target_days - 10:
        print(f"Note: Exchange has ~{have_days} days of history "
              f"(less than requested {years}y). Using all available data.")

    # Build summary / averages
    d = build_summary(df, win=90)
    stats = average_run_lengths(d["regime"])

    print("\n=== Average Bull/Bear Runs ===")
    for k, v in stats.items():
        if isinstance(v, float) and not np.isnan(v):
            print(f"{k}: {v:.2f}")
        else:
            print(f"{k}: {v}")

    # Whole-sample averages for reference
    avg_support_all    = float(d["avg_support"].mean(skipna=True))
    avg_resistance_all = float(d["avg_resistance"].mean(skipna=True))
    avg_price_all      = float(d["avg_price"].mean(skipna=True))

    print("\n=== Whole-sample Averages (levels) ===")
    print(f"Avg Support     ~ {avg_support_all:.6f}")
    print(f"Avg Resistance  ~ {avg_resistance_all:.6f}")
    print(f"Avg Price       ~ {avg_price_all:.6f}")

    # Extract multiple key levels & plot CHART 1
    levels = key_levels_from_bands(d, n_each=3)
    plot_average_chart_with_levels(d, resolved, levels)

    # Live overlay (CHART 2)
    last_close = float(d["close"].iloc[-1])
    live = fetch_live_price(exchange_id, resolved)
    if live is None:
        print("Live price not available; plotting last close only.")
    plot_live_overlay(levels, live_price=live, last_close=last_close, symbol_shown=resolved)


if __name__ == "__main__":
    main()
